In [1]:
%cd ../
%matplotlib inline

/home/eli/AnacondaProjects/HTFATorch


In [2]:
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils

import csv
import glob
import logging
import os
import re

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

db_file = 'data/synthetic_data.db'
mask_file = 'data/synthetic_data/wholebrain.nii.gz'

HIGH_CONDITION_LAST = 10
OVERWRITE = True

In [4]:
existed = os.path.isfile(db_file)
db = niidb.FMriActivationsDb(db_file, mask=mask_file)

In [5]:
if not existed or OVERWRITE:
    data_files = utils.sorted_glob('data/synthetic_data/nifti_files/*.nii')
    regex = re.compile("subject_([0-9]{1,2})_task_([1-3])_stimulus_([0-9]{1,2}).nii")
    for data_file in data_files:
        m = regex.search(data_file)
        [subject, task, stimulus] = m.groups()
        subject = int(subject)
        task = int(task)
        stimulus = int(stimulus)
        logging.info("Recording blocks in %s", data_file)
        
        if subject >= 1 and subject <= HIGH_CONDITION_LAST:
            condition = 0
        else:
            condition = 1
        
        synthetic_block = niidb.FMriActivationBlock(zscore=False)
        if task == 1:
            synthetic_block.task = 'rest_' + str(stimulus)
        elif task == 2:
            synthetic_block.task = 'baseline_' + str(stimulus)
        else:
            synthetic_block.task = 'task_' + str(stimulus)
        synthetic_block.filename = data_file
        synthetic_block.subject = subject
        synthetic_block.start_time = 0
        synthetic_block.end_time = None
        synthetic_block.individual_differences = {
            'condition': condition,
            'stimulus_group': task,
        }
        db.upsert(synthetic_block)
        logging.info('Subject %d performing task %s', subject, synthetic_block.task)

04/22/2019 13:17:40 Recording blocks in data/synthetic_data/nifti_files/subject_10_task_1_stimulus_1.nii
04/22/2019 13:17:40 Subject 10 performing task rest_1
04/22/2019 13:17:40 Recording blocks in data/synthetic_data/nifti_files/subject_10_task_1_stimulus_10.nii
04/22/2019 13:17:40 Subject 10 performing task rest_10
04/22/2019 13:17:40 Recording blocks in data/synthetic_data/nifti_files/subject_10_task_1_stimulus_11.nii
04/22/2019 13:17:40 Subject 10 performing task rest_11
04/22/2019 13:17:40 Recording blocks in data/synthetic_data/nifti_files/subject_10_task_1_stimulus_12.nii
04/22/2019 13:17:40 Subject 10 performing task rest_12
04/22/2019 13:17:40 Recording blocks in data/synthetic_data/nifti_files/subject_10_task_1_stimulus_13.nii
04/22/2019 13:17:40 Subject 10 performing task rest_13
04/22/2019 13:17:40 Recording blocks in data/synthetic_data/nifti_files/subject_10_task_1_stimulus_14.nii
04/22/2019 13:17:40 Subject 10 performing task rest_14
04/22/2019 13:17:40 Recording blocks

In [6]:
logging.info('Finished building NiiDb out of synthetic dataset')

04/22/2019 13:18:17 Finished building NiiDb out of synthetic dataset
